# Check CPU

In [2]:
!nvidia-smi

Wed Nov 20 11:55:23 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   39C    P8             20W /  170W |     952MiB /  12288MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
HOME = os.getcwd()
print(HOME)

d:\Github\football-cv-analysis\training


# Install Dependencies

In [4]:
%pip install -q ultralytics roboflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: C:\Users\jcdos\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Imports

In [5]:
from roboflow import Roboflow
from dotenv import load_dotenv
from IPython.display import Image

# Download Dataset

In [6]:
%cd {HOME}/datasets

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("roboflow-jvuqo").project("football-field-detection-f07vi")
version = project.version(15)
dataset = version.download("yolov8")

[WinError 2] The system cannot find the file specified: 'd:\\Github\\football-cv-analysis\\training/datasets'
d:\Github\football-cv-analysis\training
loading Roboflow workspace...


using bookmarks requires you to install the `pickleshare` library.


loading Roboflow project...



Extracting Dataset Version Zip to football-field-detection-15 in yolov8:: 100%|██████████| 646/646 [00:00<00:00, 2188.36it/s]


# Update data.yml

In [ ]:
%sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
%sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

# Start Training

In [ ]:
%cd {HOME}

!yolo task=pose mode=train model=yolov8x-pose.pt data={dataset.location} batch_size=16 epochs=100 imgsz=640 mosaic=0.0 plots=True

# Deploy Model

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8-pose", model_path=f"{HOME}/runs/pose/train/")